In [1]:
import sys
sys.path.insert(1, '../../libs')

from utils import get_data, temporal_train_test_split
import pandas as pd
import seaborn as sns
from sklearn.utils import shuffle
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from nltk.tokenize import TweetTokenizer
import nltk
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vinicius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = get_data("../../data/authors.csv")

In [8]:
tknzr = TweetTokenizer()

texts = data.comment.str.lower()
X = texts.apply(tknzr.tokenize)

emb = Word2Vec(X, vector_size=100, min_count=1, workers=8)

In [22]:
def embedding_transform(X, tfidf, w2v):
    docs_vectors = pd.DataFrame()
    for doc in X:
      temp = pd.DataFrame()
      for word in doc:
        try:
          word_vec = tfidf[word] * w2v.wv[word]
          temp = pd.concat([temp, pd.DataFrame([word_vec])], ignore_index = True)
        except:
          pass
      doc_vector = temp.mean()
      docs_vectors = pd.concat([docs_vectors, pd.DataFrame([doc_vector])], ignore_index = True)
    return docs_vectors

In [25]:
vect = TfidfVectorizer(tokenizer=TweetTokenizer().tokenize, token_pattern=None)
vect.fit(data.comment)
tfidf_dict = dict(zip(vect.get_feature_names_out(), vect.idf_))

In [27]:
X_emb = embedding_transform(X.iloc, tfidf_dict, emb)
X_emb

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.432714,0.032157,1.099305,0.864218,0.837875,-3.109633,0.559502,3.881276,-0.528992,-2.170392,...,2.906217,0.421129,1.411148,1.064761,1.959023,1.601933,0.523119,0.897019,-0.426631,0.103120
1,0.125898,-0.242185,0.054639,0.735327,0.204172,-0.287813,0.222079,0.502998,0.110937,-0.101309,...,0.112329,-0.015934,0.373650,0.769785,0.104709,0.382138,-0.123530,0.540850,-0.093899,-0.297840
2,-1.789684,0.469343,0.480798,1.275060,1.201183,-4.118135,0.754351,5.680285,-1.273818,-3.274171,...,2.970081,1.143064,0.718834,0.835258,3.357937,2.351601,1.128088,-0.537648,-0.405631,-0.016753
3,0.515221,-0.989140,0.209595,3.852181,0.815947,-1.457306,1.570920,3.043237,0.228163,-0.285480,...,0.800781,-0.175123,1.634715,3.569113,1.248902,2.070493,-0.804131,2.869144,-0.717174,-1.667274
4,-1.702236,0.606132,0.567707,1.606782,1.322883,-4.053792,0.866824,5.870177,-1.412498,-3.502074,...,3.181649,0.857197,0.913484,0.512868,3.121929,2.340846,1.325590,-0.597204,-0.446730,0.198760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,-2.034047,0.787279,0.574811,1.610703,1.224072,-4.229044,1.403060,5.994061,-1.946656,-3.666466,...,2.869701,0.861157,0.639911,0.795298,3.371670,2.191857,1.381457,-1.025427,-0.031438,0.329014
14516,-1.124242,0.158688,0.770907,1.060586,0.924784,-3.784168,1.347962,4.833085,-1.681440,-2.746742,...,2.604823,0.653581,0.731826,1.337521,2.676605,1.802011,0.377739,0.242872,-0.449507,0.369466
14517,-1.643308,0.389124,0.433485,1.175384,0.990874,-3.049644,0.797130,4.725774,-1.448936,-2.553259,...,2.127361,0.851902,0.370737,0.777647,2.736876,1.877308,0.973313,-0.525711,-0.142268,0.009305
14518,-1.452519,0.131700,0.448340,1.277682,0.816590,-3.764020,1.489416,5.053856,-2.616175,-3.260070,...,1.779871,0.694011,0.127503,0.732742,2.787655,1.436910,1.166911,-0.878382,0.542903,0.924574


In [71]:
df_emb = pd.concat([data.reset_index()[["username", "comment", "created_utc"]], 
                X_emb.reset_index().drop("index", axis=1)], 
                axis=1)
df_emb

,username,comment,created_utc,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,Manada_2,Levonhardu para deputado federal!,2022-03-09 14:17:46+00:00,-0.432714,0.032157,1.099305,0.864218,0.837875,-3.109633,0.559502,...,2.906217,0.421129,1.411148,1.064761,1.959023,1.601933,0.523119,0.897019,-0.426631,0.103120
1,BluePirate89,Basement dwellers,2022-03-31 16:00:06+00:00,0.125898,-0.242185,0.054639,0.735327,0.204172,-0.287813,0.222079,...,0.112329,-0.015934,0.373650,0.769785,0.104709,0.382138,-0.123530,0.540850,-0.093899,-0.297840
2,9-Cortes,Frase mentirosa. Sempre a mesma coisa. Sempre ...,2021-11-29 02:08:23+00:00,-1.789684,0.469343,0.480798,1.275060,1.201183,-4.118135,0.754351,...,2.970081,1.143064,0.718834,0.835258,3.357937,2.351601,1.128088,-0.537648,-0.405631,-0.016753
3,Supermunch2000,Forced memes are forced memes.,2021-12-21 12:01:05+00:00,0.515221,-0.989140,0.209595,3.852181,0.815947,-1.457306,1.570920,...,0.800781,-0.175123,1.634715,3.569113,1.248902,2.070493,-0.804131,2.869144,-0.717174,-1.667274
4,TheGza1,Já tem na região que vcs moram? Meu filho não ...,2021-08-10 12:49:03+00:00,-1.702236,0.606132,0.567707,1.606782,1.322883,-4.053792,0.866824,...,3.181649,0.857197,0.913484,0.512868,3.121929,2.340846,1.325590,-0.597204,-0.446730,0.198760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,9-Cortes,A questão do vício nem tinha sido levantada aq...,2022-01-13 17:44:46+00:00,-2.034047,0.787279,0.574811,1.610703,1.224072,-4.229044,1.403060,...,2.869701,0.861157,0.639911,0.795298,3.371670,2.191857,1.381457,-1.025427,-0.031438,0.329014
14516,9-Cortes,"O Lula é inocente e vcs cidadãos, são todos po...",2021-04-15 23:50:15+00:00,-1.124242,0.158688,0.770907,1.060586,0.924784,-3.784168,1.347962,...,2.604823,0.653581,0.731826,1.337521,2.676605,1.802011,0.377739,0.242872,-0.449507,0.369466
14517,Supermunch2000,Ele está acostumado a engolir a gosminha molha...,2022-01-05 16:33:10+00:00,-1.643308,0.389124,0.433485,1.175384,0.990874,-3.049644,0.797130,...,2.127361,0.851902,0.370737,0.777647,2.736876,1.877308,0.973313,-0.525711,-0.142268,0.009305
14518,BluePirate89,Bolsonaro depois de receber o dinheiro da milí...,2022-03-24 03:40:15+00:00,-1.452519,0.131700,0.448340,1.277682,0.816590,-3.764020,1.489416,...,1.779871,0.694011,0.127503,0.732742,2.787655,1.436910,1.166911,-0.878382,0.542903,0.924574


In [ ]:
df_emb.to_csv("../../data/authors_word2vec.csv")

In [2]:
df_emb = pd.read_csv("../../data/authors_word2vec.csv").drop("Unnamed: 0", axis=1)
df_emb

,username,comment,created_utc,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,Manada_2,Levonhardu para deputado federal!,2022-03-09 14:17:46+00:00,-0.432714,0.032157,1.099305,0.864218,0.837875,-3.109633,0.559502,...,2.906217,0.421129,1.411148,1.064761,1.959023,1.601933,0.523119,0.897019,-0.426631,0.103120
1,BluePirate89,Basement dwellers,2022-03-31 16:00:06+00:00,0.125898,-0.242185,0.054639,0.735328,0.204172,-0.287813,0.222079,...,0.112329,-0.015934,0.373650,0.769785,0.104709,0.382138,-0.123530,0.540850,-0.093899,-0.297840
2,9-Cortes,Frase mentirosa. Sempre a mesma coisa. Sempre ...,2021-11-29 02:08:23+00:00,-1.789684,0.469343,0.480798,1.275060,1.201183,-4.118136,0.754351,...,2.970081,1.143064,0.718834,0.835258,3.357937,2.351601,1.128088,-0.537648,-0.405631,-0.016753
3,Supermunch2000,Forced memes are forced memes.,2021-12-21 12:01:05+00:00,0.515221,-0.989140,0.209595,3.852181,0.815947,-1.457306,1.570920,...,0.800782,-0.175123,1.634715,3.569113,1.248902,2.070493,-0.804131,2.869144,-0.717174,-1.667274
4,TheGza1,Já tem na região que vcs moram? Meu filho não ...,2021-08-10 12:49:03+00:00,-1.702236,0.606132,0.567707,1.606782,1.322883,-4.053791,0.866824,...,3.181649,0.857197,0.913484,0.512868,3.121929,2.340846,1.325590,-0.597204,-0.446730,0.198760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,9-Cortes,A questão do vício nem tinha sido levantada aq...,2022-01-13 17:44:46+00:00,-2.034047,0.787279,0.574811,1.610703,1.224072,-4.229044,1.403060,...,2.869701,0.861157,0.639911,0.795298,3.371670,2.191857,1.381457,-1.025427,-0.031438,0.329014
14516,9-Cortes,"O Lula é inocente e vcs cidadãos, são todos po...",2021-04-15 23:50:15+00:00,-1.124242,0.158688,0.770907,1.060586,0.924784,-3.784168,1.347962,...,2.604823,0.653581,0.731826,1.337521,2.676605,1.802011,0.377739,0.242872,-0.449507,0.369466
14517,Supermunch2000,Ele está acostumado a engolir a gosminha molha...,2022-01-05 16:33:10+00:00,-1.643308,0.389124,0.433485,1.175384,0.990874,-3.049644,0.797130,...,2.127361,0.851902,0.370737,0.777647,2.736876,1.877308,0.973313,-0.525711,-0.142268,0.009305
14518,BluePirate89,Bolsonaro depois de receber o dinheiro da milí...,2022-03-24 03:40:15+00:00,-1.452519,0.131700,0.448340,1.277682,0.816590,-3.764020,1.489416,...,1.779872,0.694011,0.127503,0.732742,2.787655,1.436910,1.166911,-0.878382,0.542903,0.924574


In [3]:
df_emb2 = df_emb[df_emb.username.isin(["BluePirate89", "Manada_2"])]
df_emb2 = shuffle(df_emb2)

In [4]:
X = df_emb2.drop(["username", "comment"], axis=1)
y = df_emb2["username"]
df_train_test = pd.concat([X, y], axis=1)
df_train_test

,created_utc,0,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,username
9618,2022-03-26 01:05:44+00:00,-1.481745,0.947946,0.072897,1.606458,1.426401,-3.497800,0.986870,4.663478,-0.452644,...,0.860212,1.110388,0.733309,3.033682,2.289875,1.135250,-0.674533,-0.883589,-0.394019,Manada_2
11646,2022-03-21 23:47:13+00:00,-0.344231,-0.782448,0.177057,-1.619624,1.267788,-5.950239,-1.171584,2.299892,2.661003,...,1.245740,1.512144,0.914692,2.986490,2.766319,0.109046,0.319376,-2.699956,-0.354066,Manada_2
4021,2022-04-10 01:21:23+00:00,1.685461,-3.465280,-0.230147,9.096952,2.001004,-2.077495,3.441522,5.302989,0.220028,...,0.010800,3.375509,8.655671,1.297890,4.491871,-1.760944,6.971656,-1.017572,-4.311228,BluePirate89
8377,2022-04-04 15:43:35+00:00,-1.050997,0.533334,0.217901,0.652208,1.374874,-5.603068,1.265941,4.565198,-0.648751,...,1.202489,1.140063,1.453811,3.833538,3.331112,0.547708,-0.128487,-1.638251,0.023128,Manada_2
5775,2022-04-12 23:49:48+00:00,-1.371855,0.005642,0.413239,0.653451,0.828717,-5.077214,1.048367,4.628611,-1.263079,...,1.117925,0.868742,0.873109,3.165451,2.126682,0.796855,-0.625429,-0.482415,0.155583,Manada_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10829,2022-04-14 18:06:36+00:00,-1.617564,0.417447,0.522992,0.426033,1.332508,-5.173029,0.668441,5.135988,-0.637779,...,1.043151,1.001455,0.588801,3.375212,2.237561,1.095997,-0.951745,-0.551930,0.375942,Manada_2
11467,2022-04-03 21:11:10+00:00,-1.105713,0.111966,0.656196,0.846053,0.633238,-3.908080,1.070367,4.287885,-1.857374,...,0.775398,0.280167,0.817933,2.208296,1.449436,0.741022,-0.708067,0.357362,0.864915,BluePirate89
3277,2022-04-02 03:44:06+00:00,-1.726770,1.211554,0.314131,1.049185,1.397291,-3.767880,1.149607,4.899756,-1.273714,...,0.961809,0.895173,0.399160,2.982400,2.172133,1.221831,-1.044267,-0.349457,0.101517,BluePirate89
1906,2022-03-27 01:35:19+00:00,-1.310338,0.461928,0.410353,0.567193,1.245849,-3.652389,0.455469,4.068924,-0.025611,...,0.844708,1.159899,0.881020,2.547932,2.119868,0.953222,-0.352129,-0.660038,-0.105715,Manada_2


In [5]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [6]:
X_train, X_test, y_train, y_test = temporal_train_test_split(df_train_test, "BluePirate89", "Manada_2")

In [7]:
from autorship import AuthorClassifier

In [8]:
clf = AuthorClassifier(clf=LinearSVC(), embeddings=True)

In [9]:
clf.fit(X_train, y_train)

/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-Kyc7Ejcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('clf', LinearSVC())])

In [10]:
y_pred = clf.predict(X_test)

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

BluePirate89       0.90      0.87      0.88       247
    Manada_2       0.87      0.90      0.88       243

    accuracy                           0.88       490
   macro avg       0.88      0.88      0.88       490
weighted avg       0.88      0.88      0.88       490

